In [129]:
from PIL import Image as PIL_Image
import cv2
import imutils
import matplotlib.pyplot as plt
import numpy as np
import easyocr
import pytesseract
import os

In [157]:
orig_img_path = 'data/plate12.jpg'
img = cv2.imread(orig_img_path)
img = cv2.resize(img, (600,400) )
#cv2.imshow("windows", img)

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #convert to grey scale
blur_img = cv2.bilateralFilter(gray, 11, 17, 17)
cv2.imshow("windows_bl", blur_img)

ret, th1 = cv2.threshold(blur_img,165,255,cv2.THRESH_BINARY)
cv2.imshow("windows_th", th1)

kernelx = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 1))
kernely = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 3))
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 1))

morph = cv2.dilate(th1, kernelx)
morph = cv2.erode(morph, kernelx)
morph = cv2.erode(morph, kernely)
morph = cv2.dilate(morph, kernely)
#morph = cv2.erode(morph, kernel)
morph = cv2.dilate(morph, kernel)
cv2.imshow("morph", morph)


edged = cv2.Canny(gray, 30, 180) #Perform Edge detection
print(type(edged))
#cv2.imshow("windows_edged", edged)

edged1 = cv2.Canny(th1, 30, 200) #Perform Edge detection
cv2.imshow("windows_edged1", edged1)

cnts, _ = cv2.findContours(morph.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
clone = img.copy()
cv2.drawContours(clone, cnts, -1, (0, 255, 0),1)
#cv2.imshow("w", clone)

cnts = sorted(cnts, key=cv2.contourArea, reverse=True)[:10]
max_weight = 0

for item in cnts:
    x,y,weight,height = cv2.boundingRect(item)
    print(x,y,weight,height)    
    if (weight / height > 1.5) and (weight / height < 3) and(weight>max_weight):
        max_weight=weight
        crop_img = gray[y:y + height, x:x + weight]
        cv2.rectangle(clone,(x,y),(x + weight,y + height),(0,0,255),2)

cv2.imshow("w", clone)

basename = os.path.basename(orig_img_path) # basename - example.py
filename = os.path.splitext(basename)[0]  # filename - example 
saved = "result/r_" + filename + ".jpg"
saved_crop = "result/cr_" + filename + ".jpg"
cv2.imwrite(saved,clone)
#cv2.imwrite(saved,crop_img)

key = cv2.waitKey(0)
#cv2.destroyWindow("windows")
#cv2.destroyWindow("windows_gray")
cv2.destroyWindow("windows_bl")
cv2.destroyWindow("windows_th")
cv2.destroyWindow("morph")
#cv2.destroyWindow("windows_edged")
cv2.destroyWindow("windows_edged1")
cv2.destroyWindow("w")

<class 'numpy.ndarray'>
377 230 40 134
455 268 85 47
336 0 59 48
164 0 80 35
426 201 86 36
0 10 81 49
5 63 58 55
236 67 35 54
435 205 73 24
276 91 31 22
